Transfer learning using cryptobert and roberta

In [ ]:
from transformers import AutoModelForSequenceClassification
from transformers import TFAutoModelForSequenceClassification
from transformers import AutoTokenizer, AutoConfig, TextClassificationPipeline
from transformers import pipeline
import numpy as np
import pandas as pd
from scipy.special import softmax
import datetime

In [ ]:
!pip install torch

In [ ]:
!pip install protobuf==3.20.0

## Step 1 - Importing a sample of some Bitcoin Tweet Data to begin analysing the model

In [ ]:
# 20230308 - test data from twitter api, from 20220624 to 20220812 

data = pd.read_csv('~/Code/giadapi/crypto/data/raw/tweet_24-6-22_to_23-9-22.csv')

# 20230307 - test data from kaggle
# data = pd.read_csv('~/code/giadapi/crypto/data/raw/bitcoin_tweets1000000.csv', nrows = 1000)
# "\\wsl.localhost\Ubuntu\home\peter\code\giadapi\crypto\kaggle-tweets.zip"

In [ ]:
data

In [ ]:
data['datetime']

In [ ]:
#only run it if the dataset is from Twitter API
# data['text'] = data[['tweet']]
# data['date'] = data[['created_at']]

#only run if the dataset is tweets_2021_reduced.csv
data['date'] = data['datetime']
for i in range(len(data)):
    data['date'][i] = data['datetime'][i][0:10]

In [ ]:
data = data[['text', 'date']]

In [ ]:
data

## Step 2 - Cleaning the data

In [ ]:
# I have changed this to remove more information

def preprocess(text):
    new_text = []
    text = str(text)
    text = text.replace("\n", " ")
    for t in text.split(" "):
        t = '' if t.startswith('@') and len(t) > 1 else t
        t = '' if t.startswith('http') else t
        new_text.append(t)
    return " ".join(new_text)

In [ ]:
#Create a dummy data
data['process_text'] = data.text
data['negative_bert'] = data.text
data['neutral_bert'] = data.text
data['positive_bert'] = data.text

#use the preprocess_2 to clean the data
data['process_text'] = data['text'].apply(preprocess)

In [ ]:
data

## Step 3 - analyse the language and sentiments by pretrained model

In [ ]:
!pyenv local crypto

In [ ]:
#Model 2b - Use the bert model, with full scores
MODEL_bert = f"ElKulako/cryptobert"
tokenizer_bert = AutoTokenizer.from_pretrained(MODEL_bert)
tokenizer_bert.model_max_length = 512 #solve the error: RuntimeError: The expanded size of the tensor (562) must match the existing size (514) at non-singleton dimension
config_bert = AutoConfig.from_pretrained(MODEL_bert)


# PT
model_bert = AutoModelForSequenceClassification.from_pretrained(MODEL_bert)
model_bert.config.max_position_embeddings = 512

In [ ]:
def scores_bert(sample_text):
    encoded_input_bert = tokenizer_bert(sample_text, return_tensors='pt')
    output_bert = model_bert(**encoded_input_bert)
    scores_bert = output_bert[0][0].detach().numpy()
    scores_bert = softmax(scores_bert) #1st score is negative, 2nd score is netural, 3rd score is positive
    return scores_bert

In [ ]:
data['text'] = data['process_text'].apply(scores_bert)

In [ ]:
data

In [ ]:
for i in range(len(data)):    
    data['negative_bert'][i] = data['text'][i][0]
    data['neutral_bert'][i] = data['text'][i][1]
    data['positive_bert'][i] = data['text'][i][2]

In [ ]:
data = data[['date', 'process_text', 'negative_bert', 'neutral_bert','positive_bert']]

In [ ]:
data

## Step 4: Count the total number of labels/scores (positive, negative vs neutral) by date

In [ ]:
grouped_data = pd.DataFrame(data.groupby(['date'])[['negative_bert', 'neutral_bert', 'positive_bert']].sum().fillna(0).reset_index())
grouped_data = grouped_data.rename_axis("", axis="columns")

In [ ]:
grouped_data

## Step 5 - Download the data

In [ ]:
start_date = grouped_data['date'][0]
# start_date_str = datetime.datetime.strftime(start_date, "%Y-%m-%d")
start_date

In [ ]:
end_date = grouped_data['date'][len(grouped_data)-1]
# end_date_str = datetime.datetime.strftime(end_date, "%Y-%m-%d")
end_date

In [ ]:
file_name = f"{start_date}_{end_date}_twitter_comments.csv"
data.to_csv(f"~/code/giadapi/crypto/data/processed/{file_name}")

In [ ]:
file_name = f"{start_date}_{end_date}_twitter_transferlearning.csv"
grouped_data.to_csv(f"~/code/giadapi/crypto/data/processed/{file_name}")